# **Importing data and cleaning**

**Importing the dataset**

In [ ]:
#This code is from Kaggle

import kagglehub

# Download latest version
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", path)

**Importing dependancies**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
with_mask_files = os.listdir('/kaggle/input/face-mask-dataset/data/with_mask')
with_mask_files[:5]

In [ ]:
without_mask_files = os.listdir('/kaggle/input/face-mask-dataset/data/without_mask')
without_mask_files[:5]

In [ ]:
print("Length with mask folder : ", len(with_mask_files))
print("Length without mask folder : ", len(without_mask_files))

**Creating labels**

with mask => 1

without mask => 0

In [ ]:
with_mask_labels = [1] * 3725
without_mask_labels = [0] * 3828

In [ ]:
labels = with_mask_labels + without_mask_labels

In [ ]:
len(labels)

In [ ]:
labels[:5], labels[-5:], labels[3724:3726]

In [ ]:
for i in range(0, 3724):
  if labels[i] != 1:
    print("here is a mismatch : ", i)
for i in range(3725, 7552):
  if labels[i] != 0:
    print("here is a mismatch : ", i)

So the labels are correctly assigned

**Displaying the images**

In [ ]:
img = mpimg.imread('/kaggle/input/face-mask-dataset/data/with_mask/with_mask_1.jpg')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
img = mpimg.imread('/kaggle/input/face-mask-dataset/data/without_mask/without_mask_1.jpg')
imgplot = plt.imshow(img)
plt.show()

**Images processing**

1. Resize the Images
2. Convert the images to numpy arrays

In [ ]:
with_mask_path = '/kaggle/input/face-mask-dataset/data/with_mask/'

data = []
for img_file in with_mask_files:
  image = Image.open(with_mask_path + img_file)
  image = image.resize((224,224))
  image = image.convert('RGB')
  data.append(np.array(image))

In [ ]:
len(data)

In [ ]:
data[12]

In [ ]:
without_mask_path = '/kaggle/input/face-mask-dataset/data/without_mask/'

for img_file in without_mask_files:
  image = Image.open(without_mask_path + img_file)
  image = image.resize((224,224))
  image = image.convert('RGB')
  data.append(np.array(image))

In [ ]:
len(data)

In [ ]:
plt.imshow(data[3724])

In [ ]:
plt.imshow(data[3725])

In [ ]:
type(data[0])

In [ ]:
plt.imshow(data[0])

The first 3725 (0 to 3724) images of the data array are with mask and the last ones are without mask

**Creating X and Y**

In [ ]:
X = np.array(data)
Y = np.array(labels)

In [ ]:
X.shape, Y.shape

In [ ]:
plt.imshow(X[4000])

In [ ]:
Y[4000]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

**Scaling data**

In [ ]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

# **Transfer Learning model**

**Predictive System**

**Second model using MobileNet**

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
mobilenet_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model = hub.KerasLayer(mobilenet_model, input_shape=(224,224,3), trainable=False)

In [ ]:
!pip install tf_keras

In [ ]:
import tf_keras

number_of_classes = 2

model = tf_keras.Sequential([
    pretrained_model,
    tf_keras.layers.Dense(number_of_classes)
])

model.summary()

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['acc']
)

In [ ]:
model.fit(X_train_scaled, Y_train, epochs=5)

In [ ]:
score, acc = model.evaluate(X_test_scaled, Y_test)

In [ ]:
input_image_path = input('Path of the image to be predicted : ')

input_image = cv2.imread(input_image_path)

cv2_imshow(input_image)

input_image_recized = cv2.resize(input_image, (224,224))

input_image_scaled = input_image_recized / 255

input_image_reshaped = np.reshape(input_image_scaled, [1,224,224,3])

input_prediction = model.predict(input_image_reshaped)
print(input_prediction)
input_pred_label = np.argmax(input_prediction)

if input_pred_label == 0:
  print('The person is not wearing a mask')
else:
  print('The person is wearing a mask')

U can use the images in the testing_images folder to test the model

**Saving the model for further deployment**

In [ ]:
model.save("final_model", save_format="tf")

In [ ]:
import shutil

# Zip the folder
shutil.make_archive("/kaggle/working/final_model", 'zip', "/kaggle/working/final_model")